### Configuration Interaction Singles 
This tutorial is adapted from Prof. A. E. DePrince's CIS programming project [here](https://www.chem.fsu.edu/~deprince/programming_projects/cis/)

Note: we use atomic units throughout this tutorial.

At the Hartree-Fock level of theory, the N-electron wave function is approximated as an antisymmetrized product of N one-electron functions called molecular orbitals (MOs) (a Slater determinant). The simplest representation of an excited-state wave function would be a single Slater determinant comprised of a different set of molecular orbitals, but we can obtain a slightly better description of the excited state by expressing it as a linear combination of Slater determinants that differ by the ground-state configuration by one electron. What we are describing here is a configuration interaction with single excitations (CIS) wave function; the CIS wave function for the $n^{th}$ excited state can be expressed as
\begin{equation}
|\Psi_n\rangle = \sum_{ia} c_{i,a}^n |\phi_i^a\rangle,
\end{equation}
where $|\phi_i^a\rangle$ represents a Slater determinant that is singly esxcited relative to the Hartree-Fock reference determinant, and the indices $i$ and $a$ denote spin orbitals that are occupied and unoccoupied in the Hartree-Fock reference, respectively.  We can apply the Hamiltonian operator to this CIS wavefunction and project it onto a particular singly-excited determinant as follows:
\begin{equation}
\sum_{ia} \langle \phi_j^b | \hat{H} |\phi_i^a \rangle c_{i,a}^n = E_n c_{i,a}^n.
\end{equation}
We can form similar epressions for each excited state of the system, leading to an eigenvalue problem of the form:
\begin{equation}
{\bf H} {\bf c}_n = E_n {\bf c}_n,
\end{equation}
where the elements of the Hamitonian matrix elements have the form
\begin{equation}
H_{ia,jb} = \left( \epsilon_a - \epsilon_j \right) \delta_{ij} \delta_{ab} + \langle aj || ib \rangle,
\end{equation}
where $\epsilon_a$ represents the energy of Hartree-Fock orbital $a$, and $\langle aj || ib \rangle$ denotes the antisymmetrized 2-electron integral in physicist notation:
\begin{equation}
\langle aj || ib \rangle = \int d1 \: d2 \: \phi_a^*(1) \phi_j^*(2) \frac{1}{r_{12}} \phi_i(1) \phi_b(2)
- \int d1 \: d2 \: \phi_a^*(1) \phi_j^*(2) \frac{1}{r_{12}} \phi_b(1) \phi_i(2).
\end{equation}
Diagonalization of this Hamiltonian yields both singlet and triplet excited-states, but the excited determinants can be spin adapted as singlets as follows:
\begin{equation}
|\phi_i^a\rangle = \frac{1}{\sqrt{2}} \left(|\phi_{i \alpha}^{a \alpha} \rangle +  |\phi_{i \beta}^{a \beta} \rangle\right),
\end{equation}
which yields the following matrix elements for the spin-adapated Hamiltonian:
\begin{equation}
H_{ia,jb} = \left( \epsilon_a - \epsilon_j \right) \delta_{ij} \delta_{ab} + 2 \langle aj | ib \rangle - \langle aj | bi \rangle.
\end{equation}
In chemist's notation, these elements are 
\begin{equation}
H_{ia,jb} = \left( \epsilon_a - \epsilon_j \right) \delta_{ij} \delta_{ab} + 2 \left( ia | jb \right) - \left( ij |ab \right),
\end{equation}
where
\begin{equation}
\left( ia | jb \right) = \int d1 \: d2 \: \phi_i^*(1) \phi_a(1) \frac{1}{r_{12}} \phi_j^*(2) \phi_i(2).
\end{equation}

We will implement CIS in the spin-adapted basis below and compare to the results directly computed by `psi4`.

In [31]:
import psi4
import numpy as np
from psi4.driver.procrouting.response.scf_response import tdscf_excitations

Setup molecule and options in `psi4`

In [32]:
mol = psi4.geometry("""
0 1
O
H 1 1.0
H 1 1.0 2 104.5
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

Run `psi4` and save wavefunction for further analysis.  Also run the method `tdscf_excitations` 
with `tda=True` to get the CIS excitation energies from `psi4` itself.  

In [59]:
# compute the Hartree-Fock energy and wavefunction
scf_e, wfn = psi4.energy('SCF', return_wfn=True)
rpa = tdscf_excitations(wfn, states=4, tda=True)
print(rpa)

right vecs
[[ 1.11496908e-16 -5.29257371e-20]
 [-5.20723397e-17 -1.62756122e-17]
 [-8.97175766e-16  1.09154215e-16]
 [-1.54415002e-15  2.95588487e-17]
 [ 1.00000000e+00  4.76559551e-16]]
left vecs
[[ 1.11496908e-16 -5.29257371e-20]
 [-5.20723397e-17 -1.62756122e-17]
 [-8.97175766e-16  1.09154215e-16]
 [-1.54415002e-15  2.95588487e-17]
 [ 1.00000000e+00  4.76559551e-16]]
dipole
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.07919296e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.41172844e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.08620836e-18
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.07919296e-02 -6.41172844e-01  9.08620836e-18  0.00000000e+00
   0.00000000e+00 -2.87550614e-01 -2.87550614e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  

Get some basic information from the `wfn` object returned by `psi4`:

In [41]:
# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc   = wfn.nalpha()

# total number of orbitals
nmo     = wfn.nmo()

# number of virtual orbitals
nvirt   = nmo - ndocc

# orbital energies
eps     = np.asarray(wfn.epsilon_a())

# occupied orbitals:
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals:
Cv = wfn.Ca_subset("AO", "VIR")

Recall we need two types of electron repulsion integrals:
$\left( ia | jb \right)$ comprise all (occupied-virtual, occupied-virtual) integrals and
$\left( ij | ab \right)$ comprise all (occupied occupied, virtual virtual) integrals.
We will use some of the Mints Helper functions to grab these specific integral blocks below.

The MintsHelper class can construct tensors containing these specific classes of orbitals, provided we provide to it the corresponding definitions of the molecular orbitals (given by the `Co` and `Cv` matrices above):

In [42]:
# use Psi4's MintsHelper to generate ERIs
mints = psi4.core.MintsHelper(wfn.basisset())

# build the (ov|ov) integrals:
ovov = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

# build the (oo|vv) integrals:
oovv = np.asarray(mints.mo_eri(Co, Co, Cv, Cv))

We also need the orbital energies which we obtained already using `wfn.epsilon_a()`.  We will now sort them
into two different arrays: an array for the occupied orbital energies `eps_o` and an array for the virtual orbital energies `eps_v`.

In [60]:
# strip out occupied orbital energies, eps_o spans 0..ndocc-1
eps_o = eps[:ndocc]

# strip out virtual orbital energies, eps_v spans 0..nvirt-1
eps_v = eps[ndocc:]
### if you want to print these arrays, go ahead and uncomment!
#print(oovv)
#print(ovov)
#print(eps_o)
#print(eps_v)

We will make an array for the spin-adapted CIS Hamiltonian, which is an $N \times N$ matrix
with $N = n_{occ} \cdot n_{virt}$.

In [61]:
Ham = np.zeros((ndocc*nvirt,ndocc*nvirt))

Given these tensors, you can access the element $(ij|ab)$ in Python as `oovv[i,j,a,b]`. Here, the indices $i$ and $j$ run from $0$ to $ndocc-1$, and the indices $a$ and $b$ run from $0$ to $nvirt-1$.

In [62]:
for i in range(0, ndocc):
    for a in range(0, nvirt):
        ia = i*nvirt + a
        
        for j in range(0, ndocc):
            for b in range(0, nvirt):
                jb = j*nvirt + b
                term1 = eps_v[a] - eps_o[i]
                term2 = 2 * ovov[i, a, j, b] - oovv[i,j,a,b]
                ### if you want to print all the elements, uncomment below!
                #print(ia,jb, eps_v[a], eps_o[i], i, j, a, b, term1, term2)
                if (i==j) and (a == b):
                    Ham[ia, jb] = term1 + term2
                else:
                    Ham[ia, jb] = term2


Note that for the water molecule in the sto-3g basis with $r_{OH} = 1.0$ and $\theta_{HOH} = 104.5^{\circ}$, 
you can check the Hamiltonian matrix against [this one](https://www.chem.fsu.edu/~deprince/programming_projects/cis/hamiltonian.txt):

In [63]:
### uncomment to print the Hamiltonian matrix
#print(Ham)

In [66]:
# diagonalize Hamiltonian
ECIS, CCIS = np.linalg.eig(Ham)
print(ECIS[7])
print(np.reshape(CCIS[7,:],(5,2)))
print(rpa)
#[[ 1.11496908e-16 -5.29257371e-20]
# [-5.20723397e-17 -1.62756122e-17]
# [-8.97175766e-16  1.09154215e-16]
# [-1.54415002e-15  2.95588487e-17]
# [ 1.00000000e+00  4.76559551e-16]]

0.6574278638929366
[[ 2.72302117e-17 -1.50950170e-03]
 [-3.37558504e-15  2.96952119e-15]
 [ 4.56279084e-15  2.58726225e-02]
 [ 4.59842248e-01  8.87622350e-01]
 [-9.07220946e-16 -1.67583245e-15]]
[{'EXCITATION ENERGY': 0.4422030179117806, 'ELECTRIC DIPOLE TRANSITION MOMENT (LEN)': array([ 1.03986362e-01, -1.16181470e-16, -7.06077139e-17]), 'OSCILLATOR STRENGTH (LEN)': 0.003187742366517262, 'ELECTRIC DIPOLE TRANSITION MOMENT (VEL)': array([ 1.36800051e-01, -2.26083283e-16,  1.60659272e-16]), 'OSCILLATOR STRENGTH (VEL)': 0.028213668239332943, 'MAGNETIC DIPOLE TRANSITION MOMENT': array([-7.97878720e-17, -2.63099938e-01, -8.05296591e-15]), 'ROTATORY STRENGTH (LEN)': 2.2270487024408488e-17, 'ROTATORY STRENGTH (VEL)': -1.098308033465929e-16, 'SYMMETRY': 'A', 'SPIN': 'singlet', 'RIGHT EIGENVECTOR ALPHA': <psi4.core.Matrix object at 0x7f8928756590>, 'LEFT EIGENVECTOR ALPHA': <psi4.core.Matrix object at 0x7f8928756590>, 'RIGHT EIGENVECTOR BETA': <psi4.core.Matrix object at 0x7f8928756590>, 'LEFT

Validating Transition Dipole Calculations:

From the `psi4` calculations for sto-3g water with $r_{OH} = 1.0$ and $r_{HOH} = 104.5^{\circ}$, the first 4 excitation energies and transition dipole moments are as follows:

| State | Excitation Energy | $\mu_x$ | $\mu_y$ | $\mu_z$ |
| :-: | :-: | :-: | :-: | :-: |
| 1     |      0.442203017  | 1.03986362e-01    |-1.16181470e-16    | -7.06077139e-17 
| 2     |      0.510607570  | 2.88048995e-15    |-5.85019003e-15    | 1.48674703e-14 
| 3     |      0.580515287  | 5.65701382e-17    |-6.45693307e-14    | 4.41206645e-01 
| 4     |      0.657427863  | 2.37113999e-17    |3.19146848e-01    | 2.66123640e-14 
